In [84]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pycocotools.coco import COCO
import skimage.io as io
import numpy as np
import random

In [87]:
classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

colors = {
    "General trash": "red",
    "Paper": "blue",
    "Paper pack": "green",
    "Metal": "orange",
    "Glass": "purple",
    "Plastic": "cyan",
    "Styrofoam": "magenta",
    "Plastic bag": "yellow",
    "Battery": "pink",
    "Clothing": "lime"
}

In [81]:
# COCO annotation 파일 경로 및 이미지 경로 설정
annFile = '../../../dataset/train.json'
imgDir = '../../../dataset/'

In [ ]:
coco = COCO(annFile)

# 이미지와 주석 불러오기
imgIds = coco.getImgIds()

# 이미지 당 바운딩 박스 개수를 저장할 딕셔너리 (1~7개의 bbox 개수에 해당하는 이미지를 분류)
bbox_count_dict = {i: [] for i in range(1, 51)}

# 모든 이미지에 대해 바운딩 박스 개수 계산
for img_id in imgIds:
    annIds = coco.getAnnIds(imgIds=img_id, iscrowd=None)
    anns = coco.loadAnns(annIds)
    bbox_count = len(anns)
    
    # 바운딩 박스 개수가 1~50인 이미지만 저장
    if 1 <= bbox_count <= 51:
        bbox_count_dict[bbox_count].append(img_id)

# 바운딩 박스 개수 별로 5개의 이미지를 랜덤하게 선택하고 시각화
for bbox_count in range(1, 51):
    print(f"\nBounding boxes: {bbox_count}")
    
    if len(bbox_count_dict[bbox_count]) < 5:
        print(f"이미지 수가 부족합니다: {len(bbox_count_dict[bbox_count])} 개만 존재.")
    
    selected_img_ids = random.sample(bbox_count_dict[bbox_count], min(5, len(bbox_count_dict[bbox_count])))

    # 1개의 행에 5개의 이미지를 출력하기 위해 subplots 생성
    fig, axes = plt.subplots(1, 5, figsize=(20, 5))  # 1행 5열, 이미지 크기를 설정

    for i, img_id in enumerate(selected_img_ids):
        img = coco.loadImgs(img_id)[0]
        image = io.imread(imgDir + img['file_name'])

        # 해당 이미지에 대한 annotation 정보 가져오기
        annIds = coco.getAnnIds(imgIds=img['id'], iscrowd=None)
        anns = coco.loadAnns(annIds)

        # 이미지 출력
        axes[i].imshow(image)
        axes[i].axis('off')  # 축 없애기

        # 바운딩 박스 그리기 및 클래스 이름 출력
        for ann in anns:
            bbox = ann['bbox']  # 바운딩 박스 정보 [x, y, width, height]
            category_id = ann['category_id']
            category_name = coco.loadCats(category_id)[0]['name']

            # 클래스에 해당하는 색상 선택
            color = colors.get(category_name, 'white')  # 기본 색상은 'white'

            # 바운딩 박스 추가
            rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], 
                                     linewidth=2, edgecolor=color, facecolor='none')
            axes[i].add_patch(rect)

            # 클래스 이름을 바운딩 박스 위에 출력
            axes[i].text(bbox[0], bbox[1] - 10, category_name, 
                         color=color, fontsize=10, fontweight='bold', 
                         bbox=dict(facecolor='white', alpha=0.5))

    plt.tight_layout()
    plt.show()